In [1]:
import numpy as np
import math
from datetime import datetime
def jarrow_rudd_option_pricing(S, K, r, sigma, T, N):

    dt = T / N                                                                 #dt is the interval of time steps
    u = math.exp((r - sigma**2/2) *dt + sigma * math.sqrt(dt))
    d = math.exp((r - sigma**2/2) *dt - sigma * math.sqrt(dt))
    p = (1+(r * dt) - d) / (u - d)                                             #risk neutral probability
    
    option_values = [[0 for j in range(i + 1)] for i in range(N + 1)]         # this line construct an array with all 0

    # Calculate option values at expiration
    for j in range(N + 1):                                                   # loop iterations, option value at each node
        option_values[N][j] = max(S * (u ** j) * (d ** (N - j)) - K, 0)      #j is the number of steps, I calculate the value of the option at each node, approach with risk neutral

    # Backward iteration to calculate option values at previous nodes
    for i in range(N - 1, -1, -1):                                           # start from the last value and go backwards, the other minus 1 means get to the last value of i and j
        for j in range(i + 1):                                               # write 2 parameters up and down
            option_values[i][j] = (p * option_values[i + 1][j + 1] + (1 - p) * option_values[i + 1][j]) / math.exp(r * dt)    # here builds the tree, i and j are the various directions in which the tree can go
    
    call_price = option_values[0][0]                                         # value of the discounted option
    put_price = call_price + K * math.exp(-r * T) - S  # Put-Call Parity
    
    return call_price, put_price

date_format = "%m/%d/%Y"

a = datetime.strptime('01/21/2022', date_format)   #start date
b = datetime.strptime('01/19/2024', date_format)   #expiration date

delta = (b - a)

# Example of pricing with akamai stock
S = 89.06       # Stock price
K = 55         # Strike price
T = float(delta.days/365)   # Time to expiration (years)
r = 0.015       # Risk-free interest rate
sigma = 1.3439  # Volatility
N = 100         # Number of time steps

call_price, put_price = jarrow_rudd_option_pricing(S, K, r, sigma, T, N)
print("Call option price:", call_price)
print("Put option price:", put_price)


Call option price: 65.79793392962759
Put option price: 30.116825400648523


In [2]:
def crr_option_pricing(S, K, T, r, sigma, n_steps, option_type):
    dt = T / n_steps
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (1+(r * dt) - d) / (u - d)

    stock_tree = np.zeros((n_steps + 1, n_steps + 1))
    for i in range(n_steps + 1):
        for j in range(i + 1):
            stock_tree[j, i] = S * (u ** (i - j)) * (d ** j)

    option_tree = np.zeros((n_steps + 1, n_steps + 1))                        # create an array of all 0 rows n_steps + 1, same as the columns

    if option_type == 'call':
        option_tree[:, n_steps] = np.maximum(0, stock_tree[:, n_steps] - K)   # select the column N_steps and take of this all rows
    elif option_type == 'put':
        option_tree[:, n_steps] = np.maximum(0, K - stock_tree[:, n_steps])
    else:
        raise ValueError("Invalid option type. Please choose 'call' or 'put'.")

    for i in range(n_steps - 1, -1, -1):
        for j in range(i + 1):
            option_tree[j, i] = np.exp(-r * dt) * (p * option_tree[j, i + 1] + (1 - p) * option_tree[j + 1, i + 1])

    return option_tree[0, 0]

date_format = "%m/%d/%Y"

a = datetime.strptime('01/21/2022', date_format)   #start date
b = datetime.strptime('01/19/2024', date_format)   #expiration date

delta = (b - a)

# Example of pricing
S = 89.06          # Stock price
K = 55          # Strike price
T = float(delta.days/365)   # Time to expiration (years)
r = 0.015       # Risk-free interest rate
sigma = 1.4548   # Volatility
n_steps = 100   # Number of steps in the binomial tree

call_option_price = crr_option_pricing(S, K, T, r, sigma, n_steps, 'call')
put_option_price = crr_option_pricing(S, K, T, r, sigma, n_steps, 'put')
   
print(f"Call option price: {call_option_price}")      # f allows me to write both text and a variable
print(f"Put option price: {put_option_price}")    

Call option price: 68.40484339932983
Put option price: 32.72413336698879
